## Beginner GeoIPS Tutorial 

Adding Sectors, Feature and Gridline Annotators, Colormappers, and Output Formatters.


Link to these slides can be found [here](https://github.com/NRLMMD-GEOIPS/presentations)

### To get the most out of this tutorial...

[Solutions](https://github.com/NRLMMD-GEOIPS/plugin_tutorial_solutions/tree/main)

Please keep the tutorial solutions up throughout this tutorial

[Beginner Tutorial Part 2 Slides](https://docs.google.com/presentation/d/1tkdRUFjZh_AdA98G1jIjaEjXn_HjYQD3PcweVmTUmTo/edit?slide=id.g2e736e3f9ff_3_94#slide=id.g2e736e3f9ff_3_94)

Also please keep these slides up to copy/paste from here as needed. 


## System requirements

- **CPU:** 1 CPU
- **RAM:** 40GB as the notebook is written, but could use more if modified.
  Reading the entire full-disk ABI image can take up to 100GB.
- **Disk Space:** 3GB storage space.

***To see the default storage location for your system, run the cell below.***
To change the default location, change the value of `tmp_root` in the following cell.

In [ ]:
import tempfile
from pathlib import Path

# Set this to a different path if you would like to use a different location
# `tempfile.gettempdir()` returns a different location on different OS.

# tmp_root = "/use/this/path/instead"
tmp_root = Path(tempfile.gettempdir()) / "geoips_tutorial_tempdirs"

print(f"Notebook temporary storage path: {tmp_root}")

## Important notes
This notebook downloads approximately 2GB of data and produces another 1GB. It
is stored in the location reported by running the next cell.

This notebook makes an attempt at cleaning up after itself, but it is
recommended that, when done using this notebook, you check to be sure that the
directory reported by the next cell has been deleted.

## Setting up your Environment

**If you don't have an environment already set up, please follow the steps 2 and 3 of the instructions linked [here](https://nrlmmd-geoips.github.io/geoips/getting-started/installing/index.html) based on the architecture of your machine.**

You can quit following the instructions after you've activated your geoips environment via <br />
``conda activate geoips``.

Run the cell below to set up your environment. This will initialize a session-specific <br />
storage directory, add it to the global notebook environment, and add a hook that attempts <br />
to clean up the temporary storage when the notebook is closed.

❗
***Important:*** While this notebook makes an effort to clean up after itself, if you <br />
are running this notebook on your own system, ***it is advisable tomanually delete the <br />
temporary directory reported above when you are done using the notebook.***

In [ ]:
import dotenv
from IPython import get_ipython
import os

from utils import notebook_environment

# Sets up the environment and sets a global variable named `temp_dir`
notebook_environment.setup(tmp_root)

with open("./.env", "w") as env_file:
    env_file.writelines(
        [
            f"GEOIPS_TESTDATA_DIR={get_ipython().user_ns['temp_dir']}/test_data\n",
            f"GEOIPS_OUTDIRS={get_ipython().user_ns['temp_dir']}/outdirs\n",
            f"GEOIPS_PACKAGES_DIR={get_ipython().user_ns['temp_dir']}\n",
            f"GEOIPS_REBUILD_REGISTRIES=True",
            f"MY_PKG_DIR={get_ipython().user_ns['temp_dir']}/cool_plugins\n",
            "MY_PKG_NAME=cool_plugins"
        ],
    )

dotenv.load_dotenv("./.env", override=True)

if not os.path.exists(os.environ["GEOIPS_TESTDATA_DIR"]):
    os.makedirs(os.environ["GEOIPS_TESTDATA_DIR"])
if not os.path.exists(os.environ["GEOIPS_OUTDIRS"]):
    os.makedirs(os.environ["GEOIPS_OUTDIRS"])


### Tutorial Contents

We're going to show you how to extend GeoIPS by adding new plugins for five new interfaces. 

We will add:  
- One new sector 
- One new feature_annotator , one new gridline_annotator 
- Three new colormappers 
- One new output_formatter

We will also show you how to: 
- Produce a NetCDF4 file using your algorithm output 
- Read your custom NetCDF4 file back in, and produce imagery from it 

### Hands on: Outputting your new product as NetCDF 

1. Copy the Cloud Depth Imagery script to a new file 
    - **Note:** the reason to keep making new scripts rather than editing in place is that 
      you can use all of these later as regression tests for your package! 
2. Edit `clavrx.conus_netcdf.my-cloud-depth-netcdf.sh`

Update `clavrx.conus_netcdf.my-cloud-depth-netcdf.sh` with the following (we updated 
what output_formatter and file_formatter we use):

```bash
#!/bin/bash

geoips run single_source \
   $GEOIPS_TESTDATA_DIR/test_data_clavrx/data/goes16_2023101_1600/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207.level2.hdf \
   --reader_name clavrx_hdf4 \
   --product_name My-Cloud-Depth \
   --output_formatter netcdf_geoips \
   --filename_formatter geoips_netcdf_fname \
   --minimum_coverage 0 \
   --sector_list conus
ss_retval=$?

exit $((ss_retval))
```

### Running Your Script 

Activate the code block below to run your modified script.

In [ ]:
%%bash

cd $MY_PKG_DIR/tests/scripts
$MY_PKG_DIR/tests/scripts/clavrx.conus_netcdf.my-cloud-depth.netcdf.sh

This will output a bunch of log output.

If your script succeeded it will end with `INTERACTIVE: Return Value 0`.

To view your output, look for a line that says `SINGLESOURCESUCCESS`.

Open the NetCDF file to view its contents (e.g. with Panoply; shown below)

![My Cloud Depth NetCDF Output](./images/panoply-netcdf-output.png)

**Note** - For more information on the output_formatter 
you just used, run:  

In [ ]:
%%bash

geoips describe output-formatter netcdf_geoips

### A Word About Sectors

Sectors are YAML plugins that define an area of interest for plotting in GeoIPS.

**Two types:** 
- **static**: static sector of a specific area 
- **dynamic**: sectors that are generated at run-time to follow an event (e.g. 
  tropical cyclone, atmospheric river, pyro-Cb, etc.) 

For this tutorial, we will be creating a **static**  sector. 

**Static Sector Properties:** 
- **Metadata**: Contains information related to the origin of the sector. This includes 
  continent, country, state, etc.
- **Projection**: Proj projection information 
  - **a**: The radius of the earth (in meters) 
  - **lat_0**: Center latitude coordinate 
  - **lon_0**: Center longitude coordinate 
  - **proj**: String representing projection type 
  - **units**: String representing units of the 
    projection 
- **Resolution**: The size of each pixel in meters 
- **Shape**: Image shape in pixels (width + height) 
- **Center**: The center x/y coordinates of the sector 

### Example Sectors

![Example Sectors](./images/example_sectors.png)

You can run the following commands to get more information about each of the sectors 
shown here by running:

In [ ]:
%%bash

geoips describe sector australia
geoips describe sector central_america
geoips describe sector south_pole

### Generating Static Sector Properties 

To generate the properties necessary to create a static sector, you must first define 
the area you want to display (captured by the appropriate satellite), based on center 
latitude and longitude bounds. 

The easiest way to do this is to open a [Google Maps](https://www.google.com/maps/) tab, 
then right click the center of your sector to obtain the center latitude and longitude 
values. 

Once you have those center latitude and longitude values, you’re ready to create 
your custom sector. 

The next slide will display the changes you will need to make to 
create a custom conus sector plugin. 

![Google Maps CONUS](./images/google_maps_conus.png)

**Note** - For listing of sector plugins, run: 

In [ ]:
%%bash

geoips list sectors

### Hands on: Creating Custom Sectors 

Let's start by copying a sector file to edit

In [ ]:
%%bash

# Make a sectors folder and change directories into it
mkdir -pv $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/sectors/static
cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/sectors/static

# Copy a sector file from GeoIPS
geoips_path="$(pip show geoips | awk '/^Location: / {print $2 "/geoips"}')"
cp "$geoips_path/plugins/yaml/sectors/static/australia.yaml" my_conus_sector.yaml

### Generating Static Sector Properties 

Update the contents of `my_conus_sector.yaml` with the following. 

You'll notice that the metadata remains untouched, however it's important to note that <br />
this section is very helpful for displaying additional information about the sector, not <br />
only for the backend of GeoIPS, but also for people using this sector plugin.
```yaml
interface: sectors
family: area_definition_static 
name: my_conus_sector 
docstring: "My CONUS Sector" 
metadata: 
  region: 
    continent: NorthAmerica 
    country: UnitedStates 
    area: x
    subarea: x
    state: x
    city: x
spec:
  area_id: my_conus_sector 
  description: CONUS
  projection: 
    a: 6371228.0 
    lat_0: 37.0
    lon_0: -96.0
    proj: eqc
    units: m
  resolution: 
    - 3000
    - 3000
  shape: 
    height: 1000
    width: 2200
  center: [0, 0]
```


### Testing Your Custom Static Sector 

The commands you ran in the previous slide create a custom conus sector. <br />
`my_conus_sector.yaml` will be an  example plugin, showing you that you can create sectors <br />
just like `conus.yaml`, to your own specifications. 

To quickly check whether or not you like the shape and resolution of your custom sector, <br />
you can use the command line function create_sector_image . This will plot and save <br />
images containing the borders and coastlines of the inputted sectors. For example, to test <br />
your custom sector, run the following: 

In [ ]:
%%bash

geoips test sector my_conus_sector

Once completed, open the `my_conus_sector.png`  image to see what your sector will look like. <br />
It should look like this:

![My CONUS Sector](./images/my_conus_sector.png)

**Note** -  You can describe the sector plugin you just 
created by running:  

In [ ]:
%%bash

geoips describe sector my_conus_sector

### Using Your Custom Static Sector

To use `my_conus_sector.yaml` in your test script, simply replace `--sector_list conus` <br />
with `--sector_list my_conus_sector` . This change means that <br />
`clavrx.conus_annotated.my-cloud-top-height.sh` will use the sector you just created, <br />
rather than the GeoIPS conus sector we’ve been using previously. 

You'll now create a new `clavrx.my_conus_sector.my-cloud-top-height.sh` to use new <br />
my_conus_sector sector plugin. 


In [ ]:
%%bash

cd $MY_PKG_DIR/tests/scripts
cp clavrx.conus_annotated.my-cloud-top-height.sh clavrx.my_conus_sector.my-cloud-top-height.sh
vim clavrx.my_conus_sector.my-cloud-top-height.sh # or another method of editing a file (if you're using an IDE, you can just open the file and edit it)

Replace the contents of `clavrx.my_conus_sector.my-cloud-top-height.sh` with the following <br />
(the only thing changed is the sector we're using):

```bash
#!/bin/bash

geoips run single_source \
    $GEOIPS_TESTDATA_DIR/test_data_clavrx/data/goes16_2023101_1600/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207.level2.hdf \
  --reader_name clavrx_hdf4 \
  --product_name My-Cloud-Top-Height \
  --output_formatter imagery_annotated \
  --filename_formatter geoips_fname \
  --minimum_coverage 0 \
  --sector_list my_conus_sector
ss_retval=$?

exit $((ss_retval))
```

Let's run that script you just created and modified.

In [ ]:
%%bash

$MY_PKG_DIR/tests/scripts/clavrx.my_conus_sector.my-cloud-top-height.sh

The result of executing that script should look like this:

![CTH my conus sector](./images/my_conus_sector_cloud_top_height.png)

## Hands on: Creating Custom Gridline/Feature Annotators 

### Defining gridline_annotators and feature_annotators 

**gridline_annotators:**
- Describe the format of the grid lines shown in your imagery. Can control: 
- **Labels**, and which to display 
- **Lines**, such as their **color**, **linestyle**, and **linewidth** 
- **Spacing**, such as the distance between **latitude** and **longitude** labels, and 
  the grid lines that represent them. 

**feature_annotators:**
- Describe the format of the features shown in your imagery. Can control: 
- **Coastlines**, **Borders**, **States**, and **Rivers**. All of these features have 
  the same parameters: 
  - Whether they are **enabled** 
  - The **color** of the feature 
  - The **linewidth** of the feature displayed 

### Example Gridline and Feature Annotators 

![Example gridline and feature annotators](./images/example_gridline_and_feature_annotators.png)

You can describe the aforementioned plugins via:

In [ ]:
%%bash

geoips describe feature-annotator default
geoips describe gridline-annotator default

### Creating a Custom gridline_annotator 

Let's begin by setting up a directory for your gridline annotators and copy over an 
existing plugin from GeoIPS.

In [ ]:
%%bash

# Make a gridline_annotators folder and change directories into it
mkdir -pv $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/gridline_annotators
cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/gridline_annotators

# Copy a gridline_annotator file from GeoIPS
geoips_path="$(pip show geoips | awk '/^Location: / {print $2 "/geoips"}')"
cp "$geoips_path/plugins/yaml/gridline_annotators/default.yaml" tutorial.yaml

### Breaking down a gridline_annotator

The following image explains each component of a gridline_annotator plugin.

![Gridline Annotator Explained](./images/gridline_annotator_explained.png)

Now that you understand what a gridline_annotator plugin does, lets modify the plugin we <br />
copied over. Replace the contents of `tutorial.yaml` with the YAML shown below. <br />
Feel free to modify the values in this YAML if you'd like.

```yaml
interface: gridline_annotators 
family: cartopy
name: tutorial 
docstring: |
  The tutorial gridline_annotators configuration.  
  All gridline labels enabled, latitude and 
  Longitude lines colored mediumseagreen, 2.5 degree 
  spacing, 1px linewidth, and [5, 3] linestyle. 
spec:
  labels:  
    top: true
    bottom: true
    left: true
    right: true
  lines:  
    color: mediumseagreen 
    linestyle: [5, 3]
    linewidth: 1
  spacing:  
    latitude: 2.5
    longitude: 2.5
```

### Creating custom feature_annotator 

Now that we've created a custom gridline_annotator plugin, let's do the same for a <br />
feature_annotator plugin.

In [ ]:
%%bash

# Make a feature_annotators folder and change directories into it
mkdir -pv $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/feature_annotators
cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/feature_annotators

# Copy a feature_annotator file from GeoIPS
geoips_path="$(pip show geoips | awk '/^Location: / {print $2 "/geoips"}')"
cp "$geoips_path/plugins/yaml/feature_annotators/default.yaml" tutorial.yaml

### Breaking down a feature_annotator

The following image explains how a feature is set up in a feature_annotator plugin.

You'll notice that a feature can either be disabled, and if not, you must define its <br /> 
color and linewidth.

![Feature Annotator Explained](./images/feature_annotator_explained.png)

Replace the contents of `tutorial.yaml` with the following. Feel free to modify feature
values if you'd like.

```yaml
interface: feature_annotators
family: cartopy
name: tutorial 
docstring: |
  The tutorial feature annotators configuration.  
  All line types enabled. All colored  
  [midnightblue, red, yellow, cyan]. 2px coastlines, 
  2px borders, 2px states, 1px rivers .
spec:
  coastline:  
    enabled: true
    edgecolor : midnightblue 
    linewidth : 2
  borders:  
    enabled: true
    edgecolor: red
    linewidth: 2
  states:  
    enabled: true
    edgecolor: darkslategray 
    linewidth: 2
  rivers:  
    enabled: true
    edgecolor: cyan
    linewidth: 1
```

### Using Your Custom Annotators

To add the effects of these plugins to your final output, simply add the following lines <br /> 
to your test script that uses the `"imagery_annotated”` output formatter.

**Note** - If you named your custom feature_annotator or gridline_annotators with <br />
different names, replace `‘tutorial’` with such names. Don’t forget you can view your <br />
new plugins using `geoips describe`!

Let's add these changes to your bash script and run it. 

In [ ]:
%%bash

cd $MY_PKG_DIR/tests/scripts
cp clavrx.conus_annotated.my-cloud-depth.sh clavrx.conus_annotated_features_gridlines.my-cloud-depth.sh


Now, add the following lines to `clavrx.conus_annotated_features_gridlines.my-cloud-depth.sh` <br />
(the bash script you just created).

```bash
--feature_annotator tutorial \
--gridline_annotator tutorial \
```

Once you've added those, you can run the script via:

In [ ]:
%%bash

$MY_PKG_DIR/tests/scripts/clavrx.conus_annotated_features_gridlines.my-cloud-depth.sh

If you copied the YAML for the gridline and feature -annotators verbatim, the result of <br />
that script should look like this:

![Feature / Gridline annotator image](./images/feature_gridline_annotator.png)

## Colorbars and Colormaps Background and Introduction 


### Using a Matplotlib Colormap 

Matplotlib has many predefined colormaps.

The `matplotlib_linear_norm` colormapper plugin is capable of using these predefined <br />
colormaps - simply provide the name of the colormap in the `“cmap_name”` argument when <br />
you define your product plugin.

For now `matplotlib_linear_norm` is the only plugin that can directly use named <br />
colormaps (which explicitly normalizes the colors using matplotlib’s `“Normalize”` <br />
function), though others may be added in the future with additional methods of <br />
manipulating the color scales.

The image below displays a sample product plugin using matplotlib built-in colormap.

![Product using matplotlib_linear_norm](./images/product_using_mpl_linear_norm.png)

### Using an ASCII Colormap 

The `matplotlib_linear_norm` plugin can also leverage `ASCII` colormap files installed <br />
within GeoIPS, installed within a plugin package, or stored in an arbitrary location on disk. 

To use an ascii colormap, specify:
  - `cmap_source: ascii` 
  - `cmap_name: <cmap_name>`

  
Installed ASCII colormaps are stored within a plugin package under `plugins/txt/ascii_palettes/`.
  - The filename must be of the form `<cmap_name>.txt` (this is how geoips finds the given <br />
    `cmap_name`)
  - The colors are defined as RGB triplets ranging from 0 to 256 and should be formatted <br /> 
    as three columns of 256 integers. 
  - Commented lines are allowed (prefaced by “#”), to provide additional context relating <br />
    to the physical meaning of the colormap (ie, min and max expected values/units, <br />
    transition points, etc) 
  - For an example, see the [tpw_purple](https://github.com/NRLMMD-GEOIPS/geoips/blob/710b58ae82f479168e7da49dfccd8650730478d8/geoips/plugins/txt/ascii_palettes/tpw_purple.txt) colormap in the geoips package. 

If you would like to specify an arbitrary full path on disk rather than installing your <br />
ascii palette within your  plugin package, additionally specify cmap_path . This is most <br />
useful for research, development, and testing purposes. 
  - `cmap_path: <full_path>/<any_name>.txt`

`matplotlib_linear_norm` provides a number of options that can be used to customize how <br />
the colormap is used and how the colorbar is drawn. 

The following image displays a sample product using ascii palette -based colormap.

![Product using ascii colormap](./images/product_using_ascii_colormap.png)

### Using a custom GeoIPS Python-based colormapper

Color information can also be specified via a python-based GeoIPS `“colormapper”` plugin, <br />
allowing customization using specific matplotlib commands and utilities. <br />
This is the most flexible method.

The following images walk you through each part of a colormapper plugin.

![Colormapper top level](./images/colormapper_top_level.png)

![Colormapper inside call function](./images/colormapper_inside_call_function.png)

The “mpl_colors_info” return dictionary is what GeoIPS uses within the matplotlib-based <br />
utilities and output formatters in order to ensure consistent application of colors, in <br />
both the imagery and the colorbars. 

Coming up, we'll walk you through creating both GeoIPS Python-based colormapper, as well <br />
as ASCII-palette based matplotlib_linear_norm products.

![Colormapper mpl_colors_info](./images/mpl_colors_info.png)

## Hands on: Creating your own custom Python-based colormapper 

Let's begin by creating a directory to hold our colormappers and copying over an existing <br />
colormapper from GeoIPS.

In [ ]:
%%bash

# Make a colormappers folder and change directories into it
mkdir -pv $MY_PKG_DIR/$MY_PKG_NAME/plugins/modules/colormappers
cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/modules/colormappers

# Create an __init__.py file
touch __init__.py

# Copy a colormapper file from GeoIPS
geoips_path="$(pip show geoips_clavrx | awk '/^Location: / {print $2 "/geoips_clavrx"}')"
cp "$geoips_path/plugins/modules/colormappers/cmap_cldHeight.py" colorful_cloud_height.py

Now, lets update the file we copied over with new content. Replace the upper portions of <br />
`colorful_cloud_height.py` with the following python code.

```python
"""Module containing colormap for colorful cloud height product.""" 

import logging

LOG = logging.getLogger(__name__) 

interface = "colormappers" 
family = "matplotlib" 
name = "colorful_cloud_height" 


def call(data_range=[ 0, 20]):
    """Colorful cloud height colormap."""
```

Once that's complete, let's update the contents of the `call()` function to change the <br />
colormap we'll produce. Feel free to choose different colors if you'ld like.

In this case, we only modify the colors, but you just as easily could modify the transition <br />
values as well. These are all just direct matplotlib/python commands creating the desired <br />
color information.

```python
    transition_vals = [
        (min_val, 1),
        (1, 2),
        (2, 3),
        (3, 4),
        (4, 6),
        (6, 8),
        (8, 10),
        (10, 15),
        (15, max_val),
    ]
    transition_colors = [
        ("pink", "red"),
        ("paleturquoise", "teal"),
        ("plum", "rebeccapurple"),
        ("yellow", "chartreuse"),
        ("limegreen", "darkgreen"),
        ("wheat", "darkorange"),
        ("darkgray", "black"),
        ("lightgray", "silver"),
        ("lightskyblue", "deepskyblue"),
    ]
```

## Hands on: Using your new python-based colormapper in a new product

### Add custom colormapper to new product. 

Let's begin by creating a new `Cloud-Base-Python-Colors` product, using `Cloud-Height` <br />
as the product_defaults:

In [ ]:
%%bash

cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/products

Add the following product entry to your `my_clavrx_products.yaml` file. Note that it <br />
makes use of the colormapper we just put together!

```yaml
    - name: Cloud-Base-Python-Colors 
      source_names:  [clavrx] 
      docstring:  |
        CLAVR-x Colorful Cloud Base Height, 
        Using a python-based custom colormapper. 
      product_defaults:  Cloud-Height 
      spec:
        variables:  ["cld_height_base", "latitude", "longitude"] 
        colormapper: 
          plugin: 
            name: colorful_cloud_height 
            arguments: {}
```

Once that's been added, let's create a new test script to make use of our new product.

In [ ]:
%%bash

cd $MY_PKG_DIR/tests/scripts
cp clavrx.conus_annotated.my-cloud-top-height.sh clavrx.conus_annotated.cloud-base-python-colors.sh

Now that we have a new test script, let's modify it to make use of our new product. <br />
Replace the line that contains

```bash
--product_name My-Cloud-Top-Height \
```

with

```bash
--product_name Cloud-Base-Python-Colors \
```

Now, run your new script!

In [ ]:
%%bash

$MY_PKG_DIR/tests/scripts/clavrx.conus_annotated.cloud-base-python-colors.sh

You should get the following image from that script:

![Colorful Cloud Height](./images/colorful_cloud_height.png)

## Hands on: Creating a new Output Formatter

### What are output formatters? 

- Output formatters take native GeoIPS xarray data and convert it into something interpretable! 
    - NetCDF, an image, HDF5, <you name it!>
- Required arguments depend on the intent of your output!

You can see what arguments are required for each family of output formatters [here](https://github.com/NRLMMD-GEOIPS/geoips/blob/main/geoips/interfaces/module_based/output_formatters.py).

You'll notice that output formatters have few to many arguments, depending on the family <br />
they inherit from.

![Easy output formatter](./images/easy_output_formatter.png)

![Hard output formatter](./images/hard_output_formatter.png)

### Making your own output formatter

Now, let's create our own output formatter. We'll begin by creating a directory for these <br />
plugins and copying over an existing output formatter from GeoIPS.

In [ ]:
%%bash

# Make a output_formatters folder and change directories into it
mkdir -pv $MY_PKG_DIR/$MY_PKG_NAME/plugins/modules/output_formatters
cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/modules/output_formatters

# Create an __init__.py file
touch __init__.py

# Copy a output_formatter file from GeoIPS
geoips_path="$(pip show geoips | awk '/^Location: / {print $2 "/geoips"}')"
cp "$geoips_path/plugins/modules/output_formatters/netcdf_geoips.py" my_netcdf_output.py

Now that we have our own output formatter file, let's edit it.

`Step 1: Update this section` 

```python
interface = "output_formatters"
family = "xarray_data"
name = "my_netcdf_output"
```

`Step 2: Add some new attributes!`

```python
# This is inside the call() function
copy_standard_metadata (xarray_obj, prod_xarray) 
for product_name in product_names: 
    prod_xarray[product_name] = xarray_obj[product_name] 
# Let's add some additional attributes that are necessary for us 
# Add the section below! 
prod_xarray = prod_xarray. assign_attrs (
    Starring= "Richard Karn" ,
    Featuring= "Jonathan Taylor Thomas" ,
    ProducedBy= "Carmen Finestra" ,
)
```

Reminder: you can always review your plugins via `geoips describe`.

In [ ]:
%%bash

geoips describe output-formatter my_netcdf_output

### Using your new output_formatter

Let's make use of our new output_formatter by creating a new test script that uses it.

In [ ]:
%%bash

cd $MY_PKG_DIR /tests/scripts
cp clavrx.conus_netcdf.my-cloud-depth-netcdf.sh clavrx.conus_netcdf.my-cloud-top-height-my-netcdf.sh

Update the script you just created to make use of the new output formatter.

Replace the following

```bash
--output_formatter netcdf_geoips \
```

with

```bash
--output_formatter my_netcdf_output \
```

Run your new script!

In [ ]:
%%bash

$MY_PKG_DIR/tests/scripts/clavrx.conus_netcdf.my-cloud-top-height-my-netcdf.sh

If you open up the NetCDF file that this script produces, it should look something like this:

![NetCDF Output](./images/netcdf_output.png)

## A word about Readers 


### Readers Overview - Content

- GeoIPS readers return a dictionary of xarrays minimally containing the following variables:


| Common Name(s) | Required name in xarray | Data format |
| :------------: | :---------------------: | :---------: |
| Latitude, Longitude | latitude, longitude | float or int |
| Variables of Interest | **<customizable but should match <br /> names used by <br /> products!>** | float or int |
| Time of Observation | time | datetime object |
| Metadata attribute: Data Source | **source_name** | string |
| Metadata attribute: Time of First <br /> Observation | start_datetime | datetime object |
| Metadata attribute: Time of Final <br /> Observation | end_datetime | datetime object |

### Why are those bolded ones so important? 

- Those two determine how GeoIPS interfaces with your reader and product! 

![Reader Bolded Items Explained](./images/reader_bolded_items.png)

### Reader overview - Structure 

- A GeoIPS reader typically consists of the following: 
    - `The call function`<br />
      The call function is the main driver of a GeoIPS reader. It accepts the kwargs that <br />
      contain the list of files to be read, and a handful of instructions that adjust how <br />
      the reader functions. 
    - `One or several read functions`<br />
      Read functions populate xarrays with data from the files themselves. Read functions <br />
      are technically optional... but they are a best practice! 
    - `Optional utility functions`<br />
      Utility functions perform some kind of operation on inputs, typically to convert <br />
      them to a format understandable by GeoIPS 

![Reader Functions](./images/reader_functions.png)

### *call* function key points 

![Reader call function key points](./images/reader_call_function_key_points.png)

### A typical *read* function 

- Largely a *dealer’s choice* (and *you’re* the dealer!) 
    - The read function needs to open the file and read the contents (remember the minimum <br />
      content slide) into a dictionary of xarrays to be passed along to GeoIPS 
- Common challenges to be aware of: 
    - 1D variables: It’s okay if your variables are 1D... so long as *all of them* are 1D.<br /> 
      You may need to do some array manipulation to get everything even! This is a common <br />
      issue particularly with times! 
    - Seaking of times, time formatting is a common issue: TAI93… UTC… binary string... <br />
      seconds since epoch… there are a lot of ways time is reported in data! Consult the <br /> 
      user’s guide for your data to figure out what how to convert time variables to the <br />
      required datetime object format. 
    - Reading in necessary channels: GeoIPS cannot intelligently read required channels <br />
      unless you code your reader to do so. Remember your call script is invoked with the <br />
      *chans* parameter– use that information to save you and your customer’s time! 


### An Example Read Function from GMI (1) 

![GMI Read Function](./images/gmi_read_function.png)

Continued...

![GMI Read Function 2](./images/gmi_read_function2.png)

One last cool thing you can do!

![Vertical data stacking](./images/vertical_stack_data.png)

## Wrap Up

### Additional Information 

- Bonus exercises can be found in the [slide package](https://docs.google.com/presentation/d/1tkdRUFjZh_AdA98G1jIjaEjXn_HjYQD3PcweVmTUmTo/edit?slide=id.g224c8803d80_0_82#slide=id.g224c8803d80_0_82) 
    - Read that NetCDF4 file to produce imagery 
    - Change which colormap you are using to create your imagery 
- GeoIPS Slack Workspace<br />
  https://geoips.slack.com/  


### Handy Links 

- Solution repository 
    - Github: https://github.com/NRLMMD-GEOIPS/plugin_tutorial_solution  
- GeoIPS repository: https://github.com/NRLMMD-GEOIPS/geoips 